In [ ]:
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import  save_image
import torch.autograd
from torch.autograd import Variable
import os

### 读取数据并使用PCA进行降维

### PCA对ds3降维

#### 对real0降维

In [ ]:
# 读取npy文件中的内容到numpy数组中
import numpy as np
data = np.load("/new_data/yhang/GNSS/Complex/complex_ds3/complex_real0_ds3.npy")
data.shape  


In [ ]:
# 使用PCA降维
from sklearn.decomposition import PCA
pca = PCA(n_components = 1000)
pca.fit(data)
data_pca = pca.transform(data)
data_pca.shape

In [ ]:
# 重命名
ds3_real0 = data_pca
np.save("/new_data/yhang/GNSS/PCA/ds3/ds3_PCA_real0.npy", ds3_real0)

#### 对real1降维

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components = 1000)
data = np.load("/new_data/yhang/GNSS/Complex/complex_ds3/complex_real1_ds3.npy")
data.shape

In [ ]:
# 数据切割不然内存会爆
len = data.shape[0]
ds3_real10 = data[0: len // 2, :]
ds3_real11 = data[len // 2: len, :]
print(ds3_real10.shape, ds3_real11.shape)

In [ ]:
pca.fit(ds3_real10)
ds3_real10_pca = pca.transform(ds3_real10)
ds3_real10_pca.shape

In [ ]:
pca.fit(ds3_real11)
ds3_real11_pca = pca.transform(ds3_real11)
ds3_real11_pca.shape

In [ ]:
ds3_real1 = np.vstack((ds3_real10_pca, ds3_real11_pca))
print(ds3_real1.shape)


In [ ]:
np.save("/new_data/yhang/GNSS/PCA/ds3/ds3_PCA_real1.npy", ds3_real1)

#### 对spoof0降维

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components = 1000)
ds3_spoof0 = np.load("/new_data/yhang/GNSS/Complex/complex_ds3/complex_spoof0_ds3.npy")
len = ds3_spoof0.shape[0]
ds3_spoof00 = ds3_spoof0[0:len // 2, :]
ds3_spoof01 = ds3_spoof0[len // 2:, :]
pca.fit(ds3_spoof00)
ds3_spoof00 = pca.transform(ds3_spoof00)
ds3_spoof00.shape

In [ ]:
pca.fit(ds3_spoof01)
ds3_spoof01 = pca.transform(ds3_spoof01)
ds3_spoof01.shape

In [ ]:
# 合并ds3_spoof0和ds3_spoof1为一个数组
ds3_spoof0 = np.vstack((ds3_spoof00, ds3_spoof01))
ds3_spoof0.shape
np.save("/new_data/yhang/GNSS/PCA/ds3/ds3_PCA_spoof0.npy", ds3_spoof0)

In [16]:
import numpy as np

### 定义超参数

In [1]:
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
import torch.autograd
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import os
import torch

batch_size = 128 #一批128个
num_epoch = 100 #总共100批
z_dimension = 200 #噪音维度
input_dimension = 1000 #输入维度
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### 加载数据

In [2]:
spoof_data = np.load("/new_data/yhang/GNSS/PCA/PCA_ds8.npy")
clean_data = np.load("/new_data/yhang/GNSS/PCA/PCA_clean.npy")
print(spoof_data.shape, clean_data.shape)

(155000, 1000) (210000, 1000)


### 定义转换器，数据转换

In [3]:

#信号数据的处理过程
signal_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5),std=(0.5))
])

# 加入标签，将相同类别的数据放在一起
real_data = clean_data

spoof_data = spoof_data

# 创建标签，对于'real'类别，我们使用1
real_labels = np.ones((real_data.shape[0],))

# 对于'spoof'类别，我们使用0
spoof_labels = np.zeros((spoof_data.shape[0],))

# 将所有数据堆叠在一起
data = np.vstack((real_data, spoof_data))

# 同样，将所有标签堆叠在一起
labels = np.concatenate((real_labels, spoof_labels))
# 对标签进行维度修改改成(labels.shape[0], 1)
labels = labels.reshape(labels.shape[0], 1)

# 拆分数据为训练集和测试集
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.3, random_state=42)
np.save("/new_data/yhang/GNSS/TEXBAT/train/ds8/data.npy", data_train)
np.save("/new_data/yhang/GNSS/TEXBAT/train/ds8/label.npy", labels_train)
np.save("/new_data/yhang/GNSS/TEXBAT/test/ds8/data.npy", data_test)
np.save("/new_data/yhang/GNSS/TEXBAT/test/ds8/label.npy", labels_test)

In [4]:
data_train = np.load("/new_data/yhang/GNSS/TEXBAT/train/ds8/data.npy")
labels_train = np.load("/new_data/yhang/GNSS/TEXBAT/train/ds8/label.npy")
print(data_train.shape, labels_train.shape)

(255500, 1000) (255500, 1)


### 制作成数据集

In [5]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
import torch.autograd
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import os

def normalize_features(data):
    # 计算每个特征的平均值和标准差
    mean = torch.mean(data, dim=0)
    std = torch.std(data, dim=0)

    # 避免除以0
    std[std == 0] = 1

    # 归一化数据
    normalized_data = (data - mean) / std

    return normalized_data

# 将numpy数组转换为torch张量
data_torch = torch.from_numpy(data_train).float()
data_torch = normalize_features(data_torch)
labels_torch = torch.from_numpy(labels_train).float()

# 创建一个TensorDataset
dataset = TensorDataset(data_torch, labels_torch)

# 创建一个DataLoader
batch_size = batch_size  # 你可以根据需要调整这个值
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True, drop_last = True)


### 定义判别器

In [6]:
#将信号28x28展开成784，然后通过多层感知器，中间经过斜率设置为0.2的LeakyReLU激活函数，
# 最后接sigmoid激活函数得到一个0到1之间的概率进行二分类。
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator,self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(input_dimension,256),#输入特征数为1000，输出为256
            nn.LeakyReLU(0.2),#进行非线性映射
            nn.Linear(256,256),#进行一个线性映射
            nn.LeakyReLU(0.2),
            nn.Linear(256,1),
            nn.Sigmoid()#也是一个激活函数，二分类问题中，
            # sigmoid可以班实数映射到【0,1】，作为概率值，
            # 多分类用softmax函数
        )
    def forward(self, x):
        x = self.dis(x)
        return x

### 定义生成器

In [7]:
#输入一个100维的0～1之间的高斯分布，然后通过第一层线性变换将其映射到256维,
# 然后通过LeakyReLU激活函数，接着进行一个线性变换，再经过一个LeakyReLU激活函数，
# 然后经过线性变换将其变成784维，最后经过Tanh激活函数是希望生成的假的信号数据分布
# 能够在-1～1之间。
class generator(nn.Module):
    
    def __init__(self):
        super(generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dimension, 256), #用线性变换将输入映射到256维
            nn.ReLU(True),       #relu激活
            nn.Linear(256, 256), #线性变换
            nn.ReLU(True),       #relu激活
            nn.Linear(256, input_dimension), #线性变换
            nn.Tanh()            #Tanh激活使得生成数据分布在【-1,1】之间
        )
 
    def forward(self, x):
        x = self.gen(x)
        return x


### 模型训练

In [8]:
#创建对象
D = discriminator()
G = generator()
if torch.cuda.is_available():
    D = D.to(device)
    G = G.to(device)

In [9]:
#########判别器训练train#####################
#分为两部分：1、真的图像判别为真；2、假的图像判别为假
#此过程中，生成器参数不断更新
 
#首先需要定义loss的度量方式  （二分类的交叉熵）
#其次定义 优化函数,优化函数的学习率为0.0003
criterion = nn.BCELoss() #是单目标二分类交叉熵函数
d_optimizer = torch.optim.Adam(D.parameters(),lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(),lr=0.0003)

In [10]:
###########################进入训练##判别器的判断过程#####################
from torch.utils.tensorboard import SummaryWriter
# 创建一个SummaryWriter实例
writer = SummaryWriter("/home/yhang/GAN/gan-torch/board/tensorboard/GAN_TEXBAT_ds8")

update_times = 0

for epoch in range(4 * num_epoch): #进行多个epoch的训练
    for i,(signals, labels) in enumerate(dataloader):
        num_signals = signals.size(0)
        
        # view()函数作用是将一个多行的Tensor,拼接成一行
        # 第一个参数是要拼接的tensor,第二个参数是-1
        # =============================训练判别器==================
        signals = signals.view(num_signals, -1)  # 将信号展成1000维的向量
        real_signals = signals.to(device)  # 将tensor变成Variable放入计算图中
        
        real_label = labels.to(device)  # 已有的信号的lable已经放在dataloader里面了


        # 计算信号的损失
        real_out = D(real_signals)  # 将数据集信号放入判别器中
        # print("real_label.size", real_label.size())
        d_loss_real = criterion(real_out, real_label).to(device)  # 得到数据集的loss

        # 损失函数和优化
        d_loss = d_loss_real  #损失包括判真损失和判假损失
        d_optimizer.zero_grad()  # 在反向传播之前，先将梯度归0
        d_loss.backward()  # 将误差反向传播
        d_optimizer.step()  # 更新参数

        # 将损失值添加到TensorBoard
        writer.add_scalar('Loss/train_ds8_mlp', d_loss, update_times)
        update_times += 1

writer.close()   
 
        # if epoch == 0:
        #     real_images=to_img(real_img.cpu().data)
        #     save_image(real_images, './img/real_images.png')
 
        # fake_images = to_img(real_img.cpu().data)
        # save_image(fake_images, './img/fake_images-{}.png'.format(epoch+1))

In [11]:
torch.save(D.state_dict(), '/home/yhang/GAN/gan-torch/model/ds8_D_model0.pth')

#### 测试测试集上模型的表现

In [12]:
data_test = np.load("/new_data/yhang/GNSS/TEXBAT/test/ds8/data.npy")
labels_test = np.load("/new_data/yhang/GNSS/TEXBAT/test/ds8/label.npy")
print(data_test.shape, labels_test.shape)

(109500, 1000) (109500, 1)


In [13]:
D = discriminator().to(device)
D.load_state_dict(torch.load('/home/yhang/GAN/gan-torch/model/ds8_D_model0.pth'))
# 将numpy数组转换为torch张量
data_torch = torch.from_numpy(data_test).float()
data_torch = normalize_features(data_torch)
labels_torch = torch.from_numpy(labels_test).float()

# 创建一个TensorDataset
dataset = TensorDataset(data_torch, labels_torch)

# 创建一个DataLoader
batch_size = batch_size  # 你可以根据需要调整这个值
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True, drop_last = True)

In [14]:
D.eval()  # 设置模型为评估模式
correct = 0
total = 0

with torch.no_grad():  # 在评估模式下，我们不需要计算梯度
    for data, labels in dataloader:
        data, labels = data.to(device), labels.to(device)
        outputs = D(data)
        predicted = (outputs > 0.5).float()  # 使用0.5作为阈值来获取预测的标签
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy of the network on the test data: %f %%' % accuracy)

Accuracy of the network on the test data: 58.345212 %


In [11]:
###########################进入训练##判别器的判断过程#####################

from torch.utils.tensorboard import SummaryWriter
# 创建一个SummaryWriter实例
writer = SummaryWriter("/home/yhang/GAN/gan-torch/tensorboard/GAN_TEXBAT_ds8")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

update_times = 0
for epoch in range(num_epoch): #进行多个epoch的训练
    for i,(signals, labels) in enumerate(dataloader):
        num_signals = signals.size(0)
        
        # view()函数作用是将一个多行的Tensor,拼接成一行
        # 第一个参数是要拼接的tensor,第二个参数是-1
        # =============================训练判别器==================
        signals = signals.view(num_signals, -1)  # 将信号展成1000维的向量
        real_signals = signals.to(device)  # 将tensor变成Variable放入计算图中
        
        real_label = labels.to(device)  # 已有的信号的lable已经放在dataloader里面了
        fake_label = torch.zeros(num_signals).reshape(-1, 1).to(device)  # 定义假的信号的label为0
        true_label = torch.ones(num_signals).reshape(-1, 1).to(device)  # 定义真实的信号的label为1

        # 计算信号的损失
        real_out = D(real_signals)  # 将数据集信号放入判别器中
        # print("real_label.size", real_label.size())
        d_loss_real = criterion(real_out, real_label).to(device)  # 得到数据集的loss
        real_scores = real_out  # 得到真实信号的判别值，输出的值越接近1越好
 
        # 计算假的信号的损失
        z = Variable(torch.randn(num_signals, z_dimension)).to(device)  # 随机生成一些噪声
        fake_signals = G(z)  # 随机噪声放入生成网络中，生成一张假的信号
        fake_out = D(fake_signals)  # 判别器判断假的信号
        d_loss_fake = criterion(fake_out, fake_label).to(device)  # 得到假的信号的loss
        fake_scores = fake_out  # 得到假信号的判别值，对于判别器来说，假信号的损失越接近0越好
 
        # 损失函数和优化
        d_loss = d_loss_real + d_loss_fake #损失包括判真损失和判假损失
        d_optimizer.zero_grad()  # 在反向传播之前，先将梯度归0
        d_loss.backward()  # 将误差反向传播
        d_optimizer.step()  # 更新参数
        writer.add_scalar('discriminator_Loss', d_loss, update_times)
        # ==================训练生成器============================
        ################################生成网络的训练###############################
        # 原理：目的是希望生成的假的信号被判别器判断为真的信号，
        # 在此过程中，将判别器固定，将假的信号传入判别器的结果与真实的label对应，
        # 反向传播更新的参数是生成网络里面的参数，
        # 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的信号让判别器以为是真的
        # 这样就达到了对抗的目的
 
        # 计算假的信号的损失
 
        z = torch.randn(num_signals, z_dimension)  # 得到随机噪声
        z = z.to(device)
        fake_signals = G(z) #随机噪声输入到生成器中，得到假的信号
        output = D(fake_signals)  # 经过判别器得到的结果
        g_loss = criterion(output, true_label).to(device)  # 得到的假的信号与真实的信号的label的loss
 
        # bp and optimize
        g_optimizer.zero_grad()  # 梯度归0
        g_loss.backward()  # 进行反向传播
        g_optimizer.step()  # .step()一般用在反向传播后面,用于更新生成网络的参数
 
        writer.add_scalar('generator_Loss', g_loss, update_times)
        update_times += 1
 
        # if epoch == 0:
        #     real_images=to_img(real_img.cpu().data)
        #     save_image(real_images, './img/real_images.png')
 
        # fake_images = to_img(real_img.cpu().data)
        # save_image(fake_images, './img/fake_images-{}.png'.format(epoch+1))

cuda:0


In [12]:
# 保存模型
torch.save(G.state_dict(), '/home/yhang/GAN/gan-torch/model/raw_G_model_ds8.pth')
torch.save(D.state_dict(), '/home/yhang/GAN/gan-torch/model/raw_D_model_ds8.pth')

#### 测试GAN

In [25]:
data_test = np.load("/new_data/yhang/GNSS/TEXBAT/test/ds8/data.npy")
labels_test = np.load("/new_data/yhang/GNSS/TEXBAT/test/ds8/label.npy")
print(data_test.shape, labels_test.shape)

D = discriminator().to(device)
D.load_state_dict(torch.load("/home/yhang/GAN/gan-torch/model/raw_D_model_ds8.pth"))
# 将numpy数组转换为torch张量
data_torch = torch.from_numpy(data_test).float()
data_torch = normalize_features(data_torch)
labels_torch = torch.from_numpy(labels_test).float()

# 创建一个TensorDataset
dataset = TensorDataset(data_torch, labels_torch)

# 创建一个DataLoader
batch_size = batch_size  # 你可以根据需要调整这个值
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True, drop_last = True)

D.eval()  # 设置模型为评估模式
correct = 0
total = 0

with torch.no_grad():  # 在评估模式下，我们不需要计算梯度
    for data, labels in dataloader:
        data, labels = data.to(device), labels.to(device)
        outputs = D(data)
        predicted = (outputs > 0.5).float()  # 使用0.5作为阈值来获取预测的标签
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy of the network on the test data: %f %%' % accuracy)

(109500, 1000) (109500, 1)
Accuracy of the network on the test data: 58.490497 %
